# Data Cleaning for Tableau Book Sales Dashboard


Book Title and authors from Web Scraping Project: 
https://github.com/jeffreykktu/data_project/tree/main/Web%20Scraping

Final Dashbaord: 
https://public.tableau.com/profile/jeff.tu6813#!/vizhome/BookSales_16213558667360/BookSalesInsight


Objective: 
- Transform the data with updated books, feature engineered the user_submitted_review to numeric values


In [2]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv("./sales_sample_cleaned.csv")
df

,date,user_submitted_review,title,total_purchased,customer_type
0,2019-05-22,it was okay,Secrets Of R For Advanced Students,7.0,Business
1,2019-11-16,Awesome!,R For Dummies,3.0,Business
2,2019-06-27,Awesome!,R For Dummies,1.0,Individual
3,2019-11-06,Awesome!,Fundamentals of R For Beginners,3.0,Individual
4,2019-07-18,Hated it,Fundamentals of R For Beginners,NaN,Business
...,...,...,...,...,...
4995,2019-04-29,Hated it,Fundamentals of R For Beginners,NaN,Business
4996,2019-10-31,Never read a better book,R For Dummies,NaN,Business
4997,2019-10-30,I learned a lot,R For Dummies,4.0,Individual
4998,2019-08-11,it was okay,Fundamentals of R For Beginners,3.0,Business


In [5]:
df.describe()

,total_purchased
count,4282.000000
mean,4.009108
std,1.995946
min,0.000000
25%,3.000000
50%,4.000000
75%,5.000000
max,12.000000


In [6]:
df.title.value_counts()

Fundamentals of R For Beginners       1809
R For Dummies                         1630
R vs Python: An Essay                  771
Secrets Of R For Advanced Students     632
Top 10 Mistakes R Beginners Make       146
R Made Easy                             12
Name: title, dtype: int64

In [7]:
# change the book names to the updated ones
new_books = ['10% Happier', 'The 10X Rule', 'A Technique for Producing Ideas', 'The Art of Profitability', 'The Art of War', 'The Compound Effect']
authors = ['Dan Harris', 'Grant Cardone', 'James Webb Young', 'Adrian Slywotzky', 'Sun Tzu', 'Darren Hardy']
book_author = {new_books[i]: authors[i] for i in range(len(new_books))}
old_titles = list(df.title.unique())

for i in range(len(old_titles)):
    df.title = df.title.replace(old_titles[i], new_books[i])

In [8]:
book_author

{'10% Happier': 'Dan Harris',
 'The 10X Rule': 'Grant Cardone',
 'A Technique for Producing Ideas': 'James Webb Young',
 'The Art of Profitability': 'Adrian Slywotzky',
 'The Art of War': 'Sun Tzu',
 'The Compound Effect': 'Darren Hardy'}

In [9]:
# add an author column
df['author'] = df['title'].apply(lambda x: x.replace(x, book_author[x]))

In [173]:
list(df.title.unique())

['10% Happier',
 'The 10X Rule',
 'A Technique for Producing Ideas',
 'The Art of Profitability',
 'The Art of War',
 'The Compound Effect']

In [175]:
df.shape

(5000, 6)

In [176]:
# drop null 'total_purchased'
df = df.dropna(subset=['total_purchased'])

In [177]:
# change customer type, and rename to "distribution_channel"
df.customer_type.value_counts()

Business      2938
Individual    1344
Name: customer_type, dtype: int64

In [178]:
replace_dict = {"Business": "Online", "Individual": "In-Person"}
df['customer_type'] = df.customer_type.replace(replace_dict)
df = df.rename(columns={"customer_type": "distribution_channel"})

<ipython-input-178-dfcf6423d8e7>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['customer_type'] = df.customer_type.replace(replace_dict)


In [179]:
df

,date,user_submitted_review,title,total_purchased,distribution_channel,author
0,2019-05-22,it was okay,10% Happier,7.0,Online,Dan Harris
1,2019-11-16,Awesome!,The 10X Rule,3.0,Online,Grant Cardone
2,2019-06-27,Awesome!,The 10X Rule,1.0,In-Person,Grant Cardone
3,2019-11-06,Awesome!,A Technique for Producing Ideas,3.0,In-Person,James Webb Young
5,2019-01-28,Never read a better book,10% Happier,1.0,Online,Dan Harris
...,...,...,...,...,...,...
4991,2019-09-16,The author's other books were better,A Technique for Producing Ideas,4.0,In-Person,James Webb Young
4992,2019-09-07,Awesome!,A Technique for Producing Ideas,1.0,In-Person,James Webb Young
4993,2019-03-19,A lot of material was not needed,A Technique for Producing Ideas,4.0,Online,James Webb Young
4997,2019-10-30,I learned a lot,The 10X Rule,4.0,In-Person,Grant Cardone


In [180]:
# Change user_submitted_review to number ratings
df.user_submitted_review.value_counts()

Would not recommend                     417
Hated it                                403
it was okay                             399
The author's other books were better    396
OK                                      394
I learned a lot                         390
Awesome!                                387
Never read a better book                376
A lot of material was not needed        370
Name: user_submitted_review, dtype: int64

## Assuming the following rating system, 5 is the best rating:

5: 'awesome!', 'never read a better book', 'i learned a lot'

4: 'it was okay','ok'

3: "the author's other books were better", 'a lot of material was not needed

2: "would not recommend"

1: "hated it"

In [181]:
old_rating_list = list(df.user_submitted_review.unique())
old_rating_list.remove(np.nan)
old_rating_list

['it was okay',
 'Awesome!',
 'Never read a better book',
 'Hated it',
 'OK',
 "The author's other books were better",
 'A lot of material was not needed',
 'Would not recommend',
 'I learned a lot']

In [182]:
rating_dict = {}
for old_rate in old_rating_list:
    if old_rate.lower() in ['awesome!', 'never read a better book', 'i learned a lot']:
        rating_dict[old_rate] = 5
    if old_rate.lower() in ['it was okay','ok']:
        rating_dict[old_rate] = 4
    if old_rate.lower() in ["the author's other books were better", 'a lot of material was not needed']:
        rating_dict[old_rate] = 3
    if old_rate.lower() in ['would not recommend']:
        rating_dict[old_rate] = 2
    if old_rate.lower() in ["hated it"]:
        rating_dict[old_rate] = 1

rating_dict

{'it was okay': 4,
 'Awesome!': 5,
 'Never read a better book': 5,
 'Hated it': 1,
 'OK': 4,
 "The author's other books were better": 3,
 'A lot of material was not needed': 3,
 'Would not recommend': 2,
 'I learned a lot': 5}

In [183]:
df.user_submitted_review = df.user_submitted_review.replace(rating_dict)

In [184]:
df.reset_index(inplace=True)

In [186]:
df.drop('index', axis='columns')

,date,user_submitted_review,title,total_purchased,distribution_channel,author
0,2019-05-22,4.0,10% Happier,7.0,Online,Dan Harris
1,2019-11-16,5.0,The 10X Rule,3.0,Online,Grant Cardone
2,2019-06-27,5.0,The 10X Rule,1.0,In-Person,Grant Cardone
3,2019-11-06,5.0,A Technique for Producing Ideas,3.0,In-Person,James Webb Young
4,2019-01-28,5.0,10% Happier,1.0,Online,Dan Harris
...,...,...,...,...,...,...
4277,2019-09-16,3.0,A Technique for Producing Ideas,4.0,In-Person,James Webb Young
4278,2019-09-07,5.0,A Technique for Producing Ideas,1.0,In-Person,James Webb Young
4279,2019-03-19,3.0,A Technique for Producing Ideas,4.0,Online,James Webb Young
4280,2019-10-30,5.0,The 10X Rule,4.0,In-Person,Grant Cardone


In [1]:
df

NameError: name 'df' is not defined

In [191]:
df.to_csv('./book_sales_cleaned_v2.csv', index=False)